# Vulture finetune

In [1]:
from utils import *

import os

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [33]:
# Misc
LOG_DIR = './log_pt/'

# Training Parameters
test_split = 0.1
learning_rate = 0.001
num_epochs = 1000
batch_size = 256
display_step = 100

# Network Parameters
num_classes = 6
keep_rate = 1

repeat_size = 82

In [28]:
raw = pd.read_csv('./vultures.csv', header=None)

In [29]:
mean = np.array([0. for i in  range(3)])
X = []
Y = []

for i in range(len(raw)):
    
    row = np.array(raw.loc[i][~raw.loc[i].isnull()])
    data = row[0:-1].astype(float)
    label = row[-1]
    
    x_axis = data[0::3]
    y_axis = data[1::3]
    z_axis = data[2::3]
    
    x_axis = repeat_crop_data (x_axis, repeat_size)
    y_axis = repeat_crop_data (y_axis, repeat_size)
    z_axis = repeat_crop_data (z_axis, repeat_size)
    
    mean[0] += x_axis.mean()
    mean[1] += y_axis.mean()
    mean[2] += z_axis.mean()
    
    X.append(np.stack((x_axis, y_axis, z_axis)))
    Y.append(label)

mean = mean/len(raw)
    
X = np.array(X).transpose((0, 2, 1))
Y = np.array(Y)
label_names = np.unique(Y)

le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

In [105]:
# train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

In [106]:
def train_test_split(X, Y, split_rate):
    
    train_idx_overall = np.array([])
    test_idx_overall = np.array([])

    for l in np.unique(Y):

        idx = np.where(Y == l)[0]

        test_size = int(len(idx) * split_rate)

        test_choice = np.random.choice(len(idx), size=test_size, replace=False)

        train_idx = np.delete(idx, test_choice)

        test_idx = idx[test_choice]
        
        train_idx_overall = np.append(train_idx_overall, train_idx)
        
        test_idx_overall = np.append(test_idx_overall, test_idx)
        
        
    return (X[train_idx_overall.astype(int)], Y[train_idx_overall.astype(int)],
            X[test_idx_overall.astype(int)], Y[test_idx_overall.astype(int)])
    

train_x, train_y, test_x, test_y = train_test_split(X, Y, test_split)

In [9]:
train_x = np.load('./vulture_train_x.npy')
train_y = np.load('./vulture_train_y.npy')
test_x = np.load('./vulture_test_x.npy')
test_y = np.load('./vulture_test_y.npy')

In [10]:
class acc_dataset (torch.utils.data.Dataset):
    
    def __init__ (self, X, Y, mean):
        super().__init__()
        self.X = X
        self.Y = Y.astype(np.float)
        self.mean = mean
        
    def __len__ (self):
        return len(self.X)
    
    def __getitem__ (self, idx):
        
        x = self.X[idx]
        y = self.Y[idx]
        
        x = (x - self.mean).transpose(1, 0)
            
        return torch.tensor(x), torch.tensor(y).type(torch.LongTensor)

In [11]:
train_set = acc_dataset(train_x, train_y, mean)
test_set = acc_dataset(test_x, test_y, mean)

In [12]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [13]:
class MyNet (nn.Module):
    
    def __init__ (self, num_classes, input_dim, drop_rate):
        super().__init__()
        
        self.input_dim = input_dim
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc3 = nn.Linear(self.input_dim*64, 1024)
        self.dropout = nn.Dropout(p=drop_rate)
        self.fc4 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        #self.pretrained_resnet.avgpool(x)
        x = x.view(-1, self.input_dim*64)
        
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return x

In [34]:
checkpoint = torch.load('./log_pt/model_storks_plain_ep_100_step_20_dr_03_acc_96_checkpoint.pth.tar')

ft_net = MyNet(num_classes=5, input_dim=repeat_size, drop_rate=0.3).double()

ft_net.load_state_dict(checkpoint)

ft_net.fc4 = nn.Linear(1024, num_classes).double()

ft_net = ft_net.to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(ft_net.parameters(), lr=learning_rate, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=120, gamma=0.1)

In [35]:
step = 0

ft_net.train()

for epoch in range(num_epochs):
    
    exp_lr_scheduler.step()
    
    for data in train_loader:
        
        x, y = data
        
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        logits = ft_net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/len(x)
        
        if step % display_step == 0:
            print('Epoch: %d, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
            
print('Final, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (step, loss, accuracy))

Epoch: 49, Step: 100, Minibatch loss: 0.719, Minibatch accuracy: 0.726
Epoch: 99, Step: 200, Minibatch loss: 0.576, Minibatch accuracy: 0.790
Epoch: 149, Step: 300, Minibatch loss: 0.420, Minibatch accuracy: 0.871
Epoch: 199, Step: 400, Minibatch loss: 0.501, Minibatch accuracy: 0.833
Epoch: 249, Step: 500, Minibatch loss: 0.413, Minibatch accuracy: 0.871
Epoch: 299, Step: 600, Minibatch loss: 0.456, Minibatch accuracy: 0.849
Epoch: 349, Step: 700, Minibatch loss: 0.536, Minibatch accuracy: 0.817
Epoch: 399, Step: 800, Minibatch loss: 0.430, Minibatch accuracy: 0.828
Epoch: 449, Step: 900, Minibatch loss: 0.441, Minibatch accuracy: 0.833
Epoch: 499, Step: 1000, Minibatch loss: 0.466, Minibatch accuracy: 0.828
Epoch: 549, Step: 1100, Minibatch loss: 0.455, Minibatch accuracy: 0.849
Epoch: 599, Step: 1200, Minibatch loss: 0.500, Minibatch accuracy: 0.844
Epoch: 649, Step: 1300, Minibatch loss: 0.491, Minibatch accuracy: 0.817
Epoch: 699, Step: 1400, Minibatch loss: 0.438, Minibatch accur

In [36]:
ft_net.eval()

test_correct = 0

with torch.no_grad():
    
    for data in test_loader:

        test_x, test_y = data
        
        test_x, test_y = test_x.to(device), test_y.to(device)

        test_logits = ft_net(test_x)
        _, test_preds = torch.max(test_logits, 1)

        test_correct_batch = (test_preds == test_y).sum().item()

        test_correct += test_correct_batch

print('Test accuracy: %.3f' % (test_correct / len(test_set)))

Test accuracy: 0.913


In [37]:
torch.save(ft_net.state_dict(), os.path.join(LOG_DIR, 'model_%s_checkpoint.pth.tar' %
                                             'vulter_plain_ep_1000_step_120_drop_0.3_ft_acc_91'))